## Load function

In [1]:
pip install robodk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import math
from math import cos,sin,radians,atan2,pi
from robodk.robolink import *
from robodk.robomath import * 
from src.function import degtorad,radtodeg,replacedot,cleandata,variaxis,integrexaxis,\
    invxyz,shifttoolaxis,final_code,firstsector,secondsector,totextfile,extractgcode

Installing required module: PySide2 ...
Running command:
/opt/anaconda3/bin/python -m pip install --upgrade pip
...
Unable to load or install <strong>PySide2</strong>. Make sure you have internet connection and administrator privileges
Installing required module: PySide2 ...
Running command:
/opt/anaconda3/bin/python -m pip install --upgrade pip
...
Unable to load or install <strong>PySide2</strong>. Make sure you have internet connection and administrator privileges


# load gcode|

In [3]:
reg ='''
G00 X32.605 Y27.326 Z-26.817 B178.296 C44.959
G18 X32.07 Z-8.825
G01 X32.011 Z-6.826 F600.
X35.282 Y27.555 Z-6.757 B178.197 C47.062 F36.
X37.879 Y28.377 Z-6.681 B178.102 C47.586
'''

# process gcode

In [4]:
df,g = extractgcode(reg)
print(df.head())
print(g[0:5])

    x_val   y_val    z_val a_val    b_val   c_val
0  32.605  27.326  -26.817  None  178.296  44.959
1   32.07    None   -8.825  None     None    None
2  32.011    None   -6.826  None     None    None
3  35.282  27.555   -6.757  None  178.197  47.062
4  37.879  28.377   -6.681  None  178.102  47.586
['G00', 'G18', 'G01', None, None]


In [5]:
df = df.loc[:2000,:].copy()

In [6]:
clean = cleandata(df)
clean.head()

,x_val,y_val,z_val,a_val,b_val,c_val
0,32.605,27.326,-26.817,0.0,178.296,44.959
1,32.070,27.326,-8.825,0.0,178.296,44.959
2,32.011,27.326,-6.826,0.0,178.296,44.959
3,35.282,27.555,-6.757,0.0,178.197,47.062
4,37.879,28.377,-6.681,0.0,178.102,47.586


In [7]:
d = integrexaxis(clean)
d.head()

,x_new,y_new,z_new,a_new,b_new,c_new
0,3.763141,42.374935,-26.817,180.0,1.704,-135.041
1,3.384568,41.996903,-8.825,180.0,1.704,-135.041
2,3.342819,41.955214,-6.826,180.0,1.704,-135.041
3,3.861555,44.600291,-6.757,180.0,1.803,-132.938
4,4.598263,47.105507,-6.681,180.0,1.898,-132.414


In [8]:
rob = rotz(clean.iloc[0,5]*pi/180)*transl(clean.iloc[0,0],clean.iloc[0,1],clean.iloc[0,2])*roty(clean.iloc[0,4]*pi/180)
print(rob)
Fanuc = transl(d.iloc[0,0],d.iloc[0,1],d.iloc[0,2])*rotz(d.iloc[0,5]*pi/180)*roty(d.iloc[0,4]*pi/180)*rotx(d.iloc[0,3]*pi/180)
print(Fanuc)
print(rob==Fanuc)

Pose(3.763, 42.375, -26.817,  -178.796, 1.206, 135.028):
[[ -0.7072996793240822, -0.7066006051872337, 0.021041586848084718, 3.7631405260015462 ],
 [ -0.7062881369233814, 0.7076125951034472, 0.021011494289163716, 42.37493450592655 ],
 [ -0.029736026455278976, 0.0, -0.999557786588975, -26.817 ],
 [ 0.0, 0.0, 0.0, 1.0 ]]

Pose(3.763, 42.375, -26.817,  -178.796, 1.206, 135.028):
[[ -0.7072996793240823, -0.7066006051872336, 0.021041586848084635, 3.7631405260015462 ],
 [ -0.7062881369233813, 0.7076125951034473, 0.021011494289163803, 42.37493450592655 ],
 [ -0.02973602645527898, 1.2241052439090014e-16, -0.999557786588975, -26.817 ],
 [ 0.000, 0.000, 0.000, 1.000 ]]

True


In [9]:
# firstsector(d,[1,2,3,4,5,6,7,8,9],g,"stem")
# secondsector(d)

In [10]:
totextfile(d,[1,2,3,4,5,6,7,8,9],g,"stemwing no shift")

In [11]:
a = invxyz(d)
a.head()

,x_new,y_new,z_new,a_new,b_new,c_new
0,31.793,-27.326,-27.775,178.796,-1.206,-135.028
1,31.793,-27.326,-9.775,178.796,-1.206,-135.028
2,31.794,-27.326,-7.775,178.796,-1.206,-135.028
3,35.052,-27.555,-7.864,178.680,-1.228,-132.924
4,37.637,-28.377,-7.932,178.598,-1.280,-132.398


In [12]:
shift = shifttoolaxis(0,0,0,90,0,90,a)
shift.tail()

,x_new,y_new,z_new,a_new,b_new,c_new
1,31.793,-27.326,-9.775,45.054,88.296,-179.987
2,31.794,-27.326,-7.775,45.054,88.296,-179.987
3,35.052,-27.555,-7.864,42.939,88.197,-179.999
4,37.637,-28.377,-7.932,42.403,88.102,179.989
5,37.637,-28.377,-7.932,42.403,88.102,179.989


In [13]:
totextfile(shift,[1,2,3,4,5,6,9],g,"sidere")